In [ ]:
%run ./../data/load-dataset.ipynb
%run ./../doc2vec/_load-d2v-model.ipynb
%run ./../various/basic-neural-network.ipynb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
ITERS = 16

RANDOM_SEED = 0

In [ ]:
corpus, labels = df[[proc_doc_col, label_col]].T.values
X = np.array([d2v_model.dv[i] for i in range(len(corpus))])
y = np.array([np.eye(2)[int(label)] for label in labels])
train_samples, test_samples, train_labels, test_labels = train_test_split(X, y, random_state=RANDOM_SEED)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

model = Sequential()
model.add(Dense(256, input_dim=300, activation=tf.nn.relu))
model.add(Dropout(0.5))
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.5))
model.add(Dense(16, activation=tf.nn.relu))
model.add(Dense(2, activation=tf.nn.softmax))
# gradient descent optimizer
solver = SGD(learning_rate=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
# configure the learning process of the model
model.compile(optimizer=solver, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_samples, train_labels, epochs=ITERS, batch_size=32)
model.evaluate(test_samples, test_labels, verbose=True)

In [ ]:
normalize = 'true'

true_preds, true_labels = [np.argmax(pred) for pred in model.predict(test_samples)], \
                          [np.argmax(label) for label in test_labels]
ConfusionMatrixDisplay.from_predictions(true_labels, true_preds, normalize=normalize,
                                        cmap=plt.cm.Blues, display_labels=('reliable', 'unreliable'))